# **Neural Network Kaggle Exercise**
Jonathan Jones

22.06.09

In [129]:
# Libraries: 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping


In [130]:
# Mount google drive:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
# Load training data:

data = '/content/drive/MyDrive/SI/Kaggle/train.csv'

df = pd.read_csv(data)

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [132]:
# Load test data:

test_data = '/content/drive/MyDrive/SI/Kaggle/test.csv'

df_test = pd.read_csv(test_data)

df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [133]:
# convert column names to lower case:

df.columns = df.columns.str.lower()

df_test.columns = df_test.columns.str.lower()

In [134]:
# Duplicated rows:

df.duplicated().sum()

0

In [135]:
# Datatypes and dictionary conformity: 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

In [136]:
# Removing columns with less than 50% of their data: 

df.dropna(axis = 1, thresh = 45, inplace = True)

df_test.dropna(axis = 1, thresh = 45, inplace = True)

# cat = make_column_selector(dtype_include= 'object')

# cat(df)

In [137]:
# Removal of ineffectual columns:

dropped_columns = ['alley', 'miscfeature', 'masvnrtype', 'masvnrarea', 
                   'fireplacequ', 'bsmtfintype1', 'bsmtfintype2', 'garagequal', 
                   'garagecond','fence', 'bsmtqual', 'bsmtcond']

df.drop(columns = dropped_columns, inplace = True)
df_test.drop(columns = dropped_columns, inplace = True)

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 68 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   lotshape       1460 non-null   object 
 7   landcontour    1460 non-null   object 
 8   utilities      1460 non-null   object 
 9   lotconfig      1460 non-null   object 
 10  landslope      1460 non-null   object 
 11  neighborhood   1460 non-null   object 
 12  condition1     1460 non-null   object 
 13  condition2     1460 non-null   object 
 14  bldgtype       1460 non-null   object 
 15  housestyle     1460 non-null   object 
 16  overallqual    1460 non-null   int64  
 17  overallcond    1460 non-null   int64  
 18  yearbuil

In [139]:
import seaborn as sns

# Heat map to see which features are correlated with our target outcome:

correlation = df.corr().round(2)

fig, hm = plt.subplots(nrows=1, ncols =1, figsize=(20,20));
hm = sns.heatmap(correlation, cmap='rocket_r', annot=True);
hm.set_title('Correlation Heatmap\n', fontsize= 24);

In [140]:
# Finding the features that have the strongest correlation with our target:

correlated_features = df[df.columns[1:]].corr()['saleprice'][:]
correlated_features.round(2).sort_values(ascending = False)
# new_df = []

# for feature in new_df:
#   if feature > 0.5:
#     new_df.append(feature)
#   else:
#     'ignore'

saleprice        1.00
overallqual      0.79
grlivarea        0.71
garagecars       0.64
garagearea       0.62
totalbsmtsf      0.61
1stflrsf         0.61
fullbath         0.56
totrmsabvgrd     0.53
yearbuilt        0.52
yearremodadd     0.51
garageyrblt      0.49
fireplaces       0.47
bsmtfinsf1       0.39
lotfrontage      0.35
wooddecksf       0.32
2ndflrsf         0.32
openporchsf      0.32
halfbath         0.28
lotarea          0.26
bsmtfullbath     0.23
bsmtunfsf        0.21
bedroomabvgr     0.17
screenporch      0.11
poolarea         0.09
mosold           0.05
3ssnporch        0.04
bsmtfinsf2      -0.01
bsmthalfbath    -0.02
miscval         -0.02
lowqualfinsf    -0.03
yrsold          -0.03
overallcond     -0.08
mssubclass      -0.08
enclosedporch   -0.13
kitchenabvgr    -0.14
Name: saleprice, dtype: float64

In [141]:
# New data frame with features that are strongly correlated to our target (.3 correlation value or better)

new_df = df[['saleprice','overallqual',
'grlivarea',
'garagecars',
'garagearea',
'totalbsmtsf',
'1stflrsf',
'fullbath',
'totrmsabvgrd',
'yearbuilt',
'yearremodadd',
'garageyrblt',
'fireplaces']]

new_df.head()

,saleprice,overallqual,grlivarea,garagecars,garagearea,totalbsmtsf,1stflrsf,fullbath,totrmsabvgrd,yearbuilt,yearremodadd,garageyrblt,fireplaces
0,208500,7,1710,2,548,856,856,2,8,2003,2003,2003.0,0
1,181500,6,1262,2,460,1262,1262,2,6,1976,1976,1976.0,1
2,223500,7,1786,2,608,920,920,2,6,2001,2002,2001.0,1
3,140000,7,1717,3,642,756,961,1,7,1915,1970,1998.0,1
4,250000,8,2198,3,836,1145,1145,2,9,2000,2000,2000.0,1


In [142]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   saleprice     1460 non-null   int64  
 1   overallqual   1460 non-null   int64  
 2   grlivarea     1460 non-null   int64  
 3   garagecars    1460 non-null   int64  
 4   garagearea    1460 non-null   int64  
 5   totalbsmtsf   1460 non-null   int64  
 6   1stflrsf      1460 non-null   int64  
 7   fullbath      1460 non-null   int64  
 8   totrmsabvgrd  1460 non-null   int64  
 9   yearbuilt     1460 non-null   int64  
 10  yearremodadd  1460 non-null   int64  
 11  garageyrblt   1379 non-null   float64
 12  fireplaces    1460 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 148.4 KB


In [145]:
test_df = df_test[['overallqual',
'grlivarea',
'garagecars',
'garagearea',
'totalbsmtsf',
'1stflrsf',
'fullbath',
'totrmsabvgrd',
'yearbuilt',
'yearremodadd',
'garageyrblt',
'fireplaces']]

test_df.isna().sum()

overallqual      0
grlivarea        0
garagecars       1
garagearea       1
totalbsmtsf      1
1stflrsf         0
fullbath         0
totrmsabvgrd     0
yearbuilt        0
yearremodadd     0
garageyrblt     78
fireplaces       0
dtype: int64

## **Preprocessing**

In [121]:
# Set target:

X = new_df.drop(columns = 'saleprice')
y = new_df['saleprice']

# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [122]:
from sklearn.pipeline import make_pipeline
from pandas.core.arrays import categorical
from sklearn.compose import make_column_selector, make_column_transformer

# Column selection / separation by data type:
# All columns are numeric:

num_selector = make_column_selector(dtype_include= 'number')

numeric_columns = num_selector(X_train)

In [123]:
# Instantiate Scaler:
scaler = StandardScaler()

# Instantiate imputers:
mean_imputer = SimpleImputer(strategy= 'mean')

# Pipelines:
numeric_pipleine = make_pipeline(mean_imputer, scaler)

In [124]:
# Preprocessing tuples:
numeric_tuple = (numeric_pipleine, numeric_columns)

# Column transformer:
preprocessor = make_column_transformer(numeric_tuple, remainder= 'passthrough')

# Data processing:

preprocessor.fit(X_train, y_train)

X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

## **XGBoost Modeling**

In [125]:
from xgboost import XGBClassifier

# Instantiate Gradient Boosting Model:

xgb = XGBClassifier()

# Fit model on training data:

xgb.fit(X_train_processed, y_train)

XGBClassifier(objective='multi:softprob')

In [146]:
# Preprocessed testing data:

mltest_df = preprocessor.transform(new_df)

In [147]:
# Test predictions

test_predictions = xgb.predict(mltest_df)

In [160]:
test_predictions

array([ 55000,  55000,  55000, ..., 385000, 392000, 475000])

In [162]:
# Data Frame of test predictions:

# xg_predictions = pd.DataFrame({'Id': df_test['id'], 'SalePrice' : test_predictions})

# xg_predictions.head()

In [164]:
# Output file:

xg_predictions.to_csv('/content/drive/MyDrive/SI/Kaggle/jjsubmission.csv', index = False)

## **Neural Network Modeling**

In [180]:
# Import Libraries:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras import metrics
from keras.models import Sequential
from keras.layers import Dense

In [181]:
# Construct a function that takes true and predicted values as arguments
# and prints all 4 metrics:

def evaluate_model(true, pred):
  """Takes true and predicted values (arrays) and prints MAE, MSE, RMSE and R2"""
  mae = mean_absolute_error(true, pred)
  mse = mean_squared_error(true, pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(true, pred)

  print(f'MAE {mae},\n MSE {mse},\n RMSE: {rmse},\n R^2: {r2} ')

In [182]:
# Learning history plotting function:

def plot_history(history):
  """Plots the metrics of a model's learning progression or history"""

  metrics = history.history.keys()

  for metric in metrics:
    if not 'val' in metric:
      plt.plot(history.history[f'{metric}'], label = f'{metric}')
      if f'val_{metric}' in metrics:
        plt.plot(history.history[f'val_{metric}'], label = f'val_{metric}')
        plt.title(f'{metric}')
        plt.show()

In [183]:
inputshape = X_train_processed.shape[1]
inputshape

# Model instantiation:
sm = Sequential()

# First hidden layer: 
sm.add(Dense(11, activation = 'relu', input_dim = inputshape))

# Second:
sm.add(Dense(9, activation = 'relu'))

# Third:
sm.add(Dense(6, activation = 'relu'))

# Output layer:
sm.add(Dense(1, activation = 'linear'))

# Network Summary:
sm.summary()

# Accuracy used as evaluation metric:
sm.compile(optimizer= 'adam', loss = 'mse', metrics= 'mae')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 11)                143       
                                                                 
 dense_9 (Dense)             (None, 9)                 108       
                                                                 
 dense_10 (Dense)            (None, 6)                 60        
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
Total params: 318
Trainable params: 318
Non-trainable params: 0
_________________________________________________________________


In [184]:
# Model fit:

history = sm.fit(X_train_processed, y_train, 
                 validation_data = (X_test_processed, y_test),
                 epochs = 100,
                 verbose = 0)

In [188]:
# Model Evaluation:

# Predictions and model evaluation:
nntest_predicition = sm.predict(mltest_df)

evaluate_model(y_test, nntest_predicition)

ValueError: ignored